In [5]:
import geopandas as gpd
import pandas as pd

import COGCCpy
from COGCCpy import production, formation_tops
print(COGCCpy.__version__)
dir(COGCCpy)

0.0.4


['__author__',
 '__builtins__',
 '__cached__',
 '__credits__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '__version__',
 'formation_tops',
 'production',
 'tops']

In [6]:
tops = formation_tops(['05-123-43203'])
tops.df

,API,FOX HILLS BASE,NIOBRARA,SHARON SPRINGS,SUSSEX
0,0512343203,620.0,6493.0,6465.0,4296.0


In [2]:
#Well.shp from COGCC
path = r"C:\Scipts\COGCCpy\tests\wells\Wells.shp"
wells = gpd.read_file(path)

#Filter down size
wells = wells[wells.API_County == '061']
wells.reset_index(inplace=True,drop=True)

print('Post filter:',wells.shape)
wells.head()

(989, 37)


,API,API_County,API_Seq,API_Label,Operat_Num,Operator,Well_Num,Well_Name,Well_Title,Citing_Typ,...,Qtr_Qtr,Section,Township,Range,Meridian,Latitude,Longitude,Utm_X,Utm_Y,geometry
0,06105000,061,05000,05-061-05000,19160,CONOCO PHILLIPS COMPANY,1,GENTZ,1 GENTZ,ACTUAL,...,SWSW,32,20S,46W,6,38.268444,-102.598958,710049,4238327,POINT (710049.000 4238327.000)
1,06105001,061,05001,05-061-05001,100705,SNEE AND EBERLY,1,JAY BOTHWELL,1 JAY BOTHWELL,ACTUAL,...,SWSW,31,20S,42W,6,38.270135,-102.174595,747174,4239564,POINT (747174.000 4239564.000)
2,06105002,061,05002,05-061-05002,74758,RISING* KEITH L.,1,PRINGLE,1 PRINGLE,ACTUAL,...,NWSE,31,20S,47W,6,38.272563,-102.717532,699663,4238521,POINT (699663.000 4238521.000)
3,06105003,061,05003,05-061-05003,1812,AMERICAN EXPLORATION CO,1,BAUGHMAN UNIT,1 BAUGHMAN UNIT,ACTUAL,...,NESW,35,20S,48W,6,38.272233,-102.759163,696022,4238396,POINT (696022.000 4238396.000)
4,06105004,061,05004,05-061-05004,100127,DOMINION OKLAHOMA TEXAS EXPLORATION & PRODUCTION,1,ERAMUSPE-STATE A,1 ERAMUSPE-STATE A,ACTUAL,...,SWNE,34,20S,48W,6,38.276283,-102.772623,694833,4238817,POINT (694833.000 4238817.000)


In [3]:
i = 0
df = pd.DataFrame()
while i < wells.shape[0]:
    i+=10 #Number of wells to process at one time
    if i < (wells.shape[0]-1):
        api_list = wells.loc[i-10:i]['API_Label'].to_list()
    else:
        api_list = wells.loc[i-10:]['API_Label'].to_list()
    
    try:
        temp = formation_tops(api_list).df
    except Exception as e:
        print(i,e)

    df = pd.concat([df,temp],ignore_index=True)
    if i%50 == 0:
        df.to_csv('061_Tops.csv',index=False)
    
    #Progress Tracker
    cmp = str(round(i/wells.shape[0]*100,2))+'% complete      '
    print('Total:',cmp)

df.to_csv('061_Tops.csv',index=False)

In [4]:
df.head()

,API,ARBUCKLE,BLAINE,CHEROKEE K,DAKOTA,LANSING-KANSAS CITY,MARMATON,MISSISSIPPIAN,MORRISON,PENNSYLVANIAN,...,MORROW V-5,MORROW-UPPER,UNKNOWN,LOWER LEWIS,LANSING-PENNSYLVANIAN,LAKOTA-DAKOTA-MUDDY,NACIMIENTO,FRUITLAND,BARKER CREEK,GRANITE
0,0506105000,5399.0,1518.0,4442.0,503.0,3590.0,4104.0,4957.0,921.0,2905.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0506105001,5562.0,NaN,4355.0,1060.0,NaN,4263.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0506105002,NaN,NaN,4256.0,NaN,NaN,4105.0,5013.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0506105003,NaN,NaN,4188.0,NaN,NaN,4032.0,4930.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0506105004,NaN,1672.0,4328.0,490.0,NaN,3994.0,4919.0,764.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
